In [5]:
import numpy as np 
import pandas as pd 
import datetime
from scipy.stats import beta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

##### Overview of the dataset

In [30]:
data = pd.read_csv("/kaggle/input/ab-testing-dataset/ab_data.csv")
print("data shape:", data.shape)
print(data.info())
data.head()

data shape: (294478, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB
None


,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


#### Data Cleaning
1. Checking for missing values

In [31]:
data.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

##### Summarizing the **group** and **landing_page** in a simple cross-tabulation table.
The table shows the total number of new_page and old_page users in both the control and Treatment. 
There is a total of 294478 users, however I shall clean the dataset further.

In [32]:
pd.crosstab(data["group"], data["landing_page"], margins=True, margins_name="Total")

landing_page,new_page,old_page,Total
group,,,
control,1928,145274,147202
treatment,145311,1965,147276
Total,147239,147239,294478


##### More information about the dataset
We can see the number of unique users, duration of the data collected, the lanfing pages to compare and the Percentage of users bucketed to both the control and treatment group is about 50%, or half for both.


In [33]:
start = datetime.datetime.strptime(data["timestamp"].min(),"%Y-%m-%d %H:%M:%S.%f")
end = datetime.datetime.strptime(data["timestamp"].max(),"%Y-%m-%d %H:%M:%S.%f")
data_duration = (end - start).days

print(f'Number of unique users in the experiment: {data["user_id"].nunique()}')
print(f'Data collected for {data_duration} days')
print(f'Landing pages to compare: {data["landing_page"].unique().tolist()}')
print(f'Percentage of users in control: {round(data[data["group"]=="control"].shape[0] * 100/data.shape[0])}%')

Number of unique users in the experiment: 290584
Data collected for 21 days
Landing pages to compare: ['old_page', 'new_page']
Percentage of users in control: 50%


Here I shall count the number of users that have been **double exposed** to both the **new_page** and **old_page** and we cann see that there are 3894 double exposed users, maybe due to technical glitch or something.

In [34]:
count=  data["user_id"].value_counts()
print((count > 1).value_counts())

False    286690
True       3894
Name: user_id, dtype: int64


##### Removing the double exposed users
To avoid sampling the same users twice and that they are quite small compared to the large number of users in the dataset.

In [35]:
valid_users = pd.DataFrame(count[count==1].index, columns=["user_id"])
data =  data.merge(valid_users, on = ["user_id"])
print(f"The updated dataset has {data.shape[0]} entries")

The updated dataset has 286690 entries


##### Adding a week column
to see and track the results of the experiment on a weekly basis

In [36]:
data["week"] =  data["timestamp"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f").isocalendar()[1])
data.head()

,user_id,timestamp,group,landing_page,converted,week
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,3
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,2
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,2
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,3


Counting the number of users on each week

In [37]:
data["week"].value_counts()

2    91380
3    91056
1    83745
4    20509
Name: week, dtype: int64

### Experiment: Bayesian Approach


##### Using the first week of control data as the prior
This may not be the case always in actual working data for companies as the prior information is always provided before the experiment.

In [38]:
prior = data[(data["week"] ==1) & (data["group"] =="control")]

##### Sampling 10000 samples from the prior data
and Determing the Purchase Convertion rate of each of the samples, in this case I'll be showing 10 Purchase Conversion rates.

In [39]:
prior_means = []
for i in range(10000):
    prior_means.append(prior.sample(10000)["converted"].mean())
    
    

In [40]:
prior_means[:10]

[0.1173, 0.1197, 0.1221, 0.123, 0.117, 0.1208, 0.1197, 0.1212, 0.1189, 0.1173]

##### Model Beta distribution from the Prior means (Distribution constructed before seeing the data).


In [42]:
prior_alpha, prior_beta, _, _ =  beta.fit(prior_means, floc=0, fscale=1)

Conducting the experiment by varying the number of weeks so as to get the experiment results at weekly points in time.
**Note that the first week of data is only for prior**.

We can see the the lift or the difference in Purchase convertion between the treatment and control group is **0.009**, with the control being slightly higher that the treatment.

In [43]:
num_weeks = 2
experiment_data = data[(data["week"] > 1) & (data["week"] <= num_weeks)]
control = experiment_data[experiment_data["group"] == "control"]
treatment = experiment_data[experiment_data["group"] == "treatment"]

# Calculate Purchase Conversion rate in percentage

control_conversion =  round(control["converted"].sum() * 100/ control["converted"].count(), 4)
treatment_conversion = round(treatment["converted"].sum() * 100/ treatment["converted"].count(), 4)

# Getting the Lift

lift = round((treatment_conversion - control_conversion) / control_conversion, 4)

print(f"Treatment Purchase Convertion rate: {treatment_conversion}%")
print(f"Control Purchase Convertion rate: {control_conversion}%")
print(f"Lift = {lift}%")

Treatment Purchase Convertion rate: 11.8175%
Control Purchase Convertion rate: 11.9251%
Lift = -0.009%


#### Creating a Prosterior distribution (Distribution construction after seeing the data) using the Priors.
 Additionally, the parameters:
1. alpha = converted users
2. beta =  non_converted

 We can see that the probability (treatment > control Purchase conversion rate ) that we are seeing a lift of **-0.009** is **32.02%**
 
 Hence we are around **70%** confidence that the Purchase Conversion rate of control > treatment by the lift.
 
**Note that the experiment results is for week 2, one can change the number of weeks, run the experiment multiple times and get new results**.

In [45]:
control_converted = control["converted"].sum()
treatment_converted = treatment["converted"].sum()
control_non_converted = control["converted"].count() - control_converted
treatment_non_converted = treatment["converted"].count() - treatment_converted

# Updating the parameters accordingly

posterior_control = beta(prior_alpha + control_converted, prior_beta + control_non_converted)
posterior_treatment = beta(prior_alpha + treatment_converted, prior_beta + treatment_non_converted)

# Sampling from Prosteriors for both control and treatment groups
control_samples = posterior_control.rvs(10000)
treatment_samples = posterior_treatment.rvs(10000)
probability = np.mean(treatment_samples > control_samples )
print(f"Probability that treatment > control: {probability * 100}%")

Probability that treatment > control: 32.019999999999996%


##### Computing Variance and mean for both groups


In [46]:
(control_mu), (control_var) = posterior_control.stats()
(treatment_mu), (treatment_var) = posterior_treatment.stats()
print(f"Control Posterior: Mean: {control_mu}, Variance: {control_var}")
print(f"Treatment Poaterior: Mean: {treatment_mu}, Variance: {treatment_var}")

Control Posterior: Mean: 0.11924373520981418, Variance: 1.7885611197750057e-06
Treatment Poaterior: Mean: 0.11840612057725858, Variance: 1.7773838847896593e-06


#### Conducting the experiment for 3 weeks (Increasing the Number of Experiment data)
We can clearly see that by changing the time period or the number of weeks, we get different results for the Purchase Conversion rates of both groups, and that still the Control Purchase Conversion rate is higher than that for the Treatment group.

The probability (treatment > control Purchase conversion rate) increases negatively **-0.011** compared to the previous week results.

Hence we are more confident that the Control Purchase Conversion rate > that that for the treatment group, around **80%** confident.

The variances of both the **Control Posterior** and the **Treatment Posterior** distributions reduces from the previous week results.
This means that the distributions are getting narrower and more distinct hence making it easier to even quantify small differences.

In [47]:
num_weeks = 3
experiment_data = data[(data["week"] > 1) & (data["week"] <= num_weeks)]
control = experiment_data[experiment_data["group"] == "control"]
treatment = experiment_data[experiment_data["group"] == "treatment"]

# Calculate Purchase Conversion rate in percentage

control_conversion =  round(control["converted"].sum() * 100/ control["converted"].count(), 4)
treatment_conversion = round(treatment["converted"].sum() * 100/ treatment["converted"].count(), 4)

# Getting the Lift

lift = round((treatment_conversion - control_conversion) / control_conversion, 4)

print(f"Treatment Purchase Convertion rate: {treatment_conversion}%")
print(f"Control Purchase Convertion rate: {control_conversion}%")
print(f"Lift = {lift}%")

Treatment Purchase Convertion rate: 11.893%
Control Purchase Convertion rate: 12.0255%
Lift = -0.011%


In [48]:
control_converted = control["converted"].sum()
treatment_converted = treatment["converted"].sum()
control_non_converted = control["converted"].count() - control_converted
treatment_non_converted = treatment["converted"].count() - treatment_converted

# Updating the parameters accordingly

posterior_control = beta(prior_alpha + control_converted, prior_beta + control_non_converted)
posterior_treatment = beta(prior_alpha + treatment_converted, prior_beta + treatment_non_converted)

# Sampling from Posteriors for both control and treatment groups
control_samples = posterior_control.rvs(10000)
treatment_samples = posterior_treatment.rvs(10000)
probability = np.mean(treatment_samples > control_samples )
print(f"Probability that treatment > control: {probability * 100}%")

Probability that treatment > control: 20.07%


In [50]:
(control_mu), (control_var) = posterior_control.stats()
(treatment_mu), (treatment_var) = posterior_treatment.stats()
print(f"Control Posterior: Mean: {control_mu}, Variance: {control_var}")
print(f"Treatment Posterior: Mean: {treatment_mu}, Variance: {treatment_var}")

Control Posterior: Mean: 0.12012500562507394, Variance: 1.0137810604759366e-06
Treatment Posterior: Mean: 0.11896623349650229, Variance: 1.0054204168087042e-06
